In [1]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
import numpy as np

start_time = datetime(2021, 4, 3, 11, tzinfo=tzlocal())

nwbfile = NWBFile(session_description = 'Example Neuropixels data saved by Open Ephys GUI', 
                  identifier = '98217279', 
                  session_start_time = start_time,
                  file_create_date = datetime.now(tzlocal()),
                  timestamps_reference_time = datetime.now(tzlocal()),
                  experimenter='joshs',
                  institution='Allen Institute',
                  experiment_description='This is just a test',
                  session_id='0x0218AF')

In [2]:
names = ['ProbeA', 'ProbeB']
locations = ['VISp', 'VISal']

electrode_idx = 0

for i in range(len(names)):

    probe = nwbfile.create_device(name=names[i],
                                  description='Neuropixels 1.0',
                                  manufacturer='imec')

    electrode_group = nwbfile.create_electrode_group(names[i] + '-AP',
                                                     description='Neuropixels 1.0 Shank',
                                                     location=locations[i],
                                                     device=probe)


    for electrode in range(384):

        x = electrode % 2
        y = electrode // 2

        nwbfile.add_electrode(id=electrode_idx,
                              x=float(x), y=float(y), z=-1.0,
                              imp = -1.0,
                              location= 'unknown',
                              filtering='High pass at 300 Hz',
                              group=electrode_group)

        electrode_idx += 1;

    electrode_group = nwbfile.create_electrode_group(names[i] + '-LFP',
                                                     description='Neuropixels 1.0 Shank',
                                                     location=locations[i],
                                                     device=probe)

    for electrode in range(384):

        x = electrode % 2
        y = electrode // 2

        nwbfile.add_electrode(id=electrode_idx,
                              x=float(x), y=float(y), z=-1.0,
                              imp = -1.0,
                              location= 'unknown',
                              filtering='Low pass at 1000 Hz',
                              group=electrode_group)

        electrode_idx += 1;

In [3]:
from pynwb.ecephys import ElectricalSeries

probeA_AP_region = nwbfile.create_electrode_table_region(tuple(np.arange(0,384)),
                                                              description='ProbeA-AP')

probeA_LFP_region = nwbfile.create_electrode_table_region(tuple(np.arange(384,768)),
                                                              description='ProbeA-LFP')

probeB_AP_region = nwbfile.create_electrode_table_region(tuple(np.arange(768,1152)),
                                                              description='ProbeB-AP')

probeB_LFP_region = nwbfile.create_electrode_table_region(tuple(np.arange(1152,1536)),
                                                              description='ProbeB-LFP')

num_samples = 1200
AP_sample_rate = 30000
LFP_sample_rate = 2500

nwbfile.add_acquisition(ElectricalSeries('Neuropix-PXI-100.ProbeA-AP',
                            data = np.zeros((num_samples,384)),
                            electrodes = probeA_AP_region,
                            timestamps=np.linspace(0,num_samples / AP_sample_rate, num_samples),
                            resolution=1/AP_sample_rate,
                            description="Probe A AP data"))

nwbfile.add_acquisition(ElectricalSeries('Neuropix-PXI-100.ProbeA-LFP',
                            data = np.zeros((num_samples // 12,384)),
                            electrodes = probeA_LFP_region,
                            timestamps=np.linspace(0,num_samples / LFP_sample_rate, num_samples),
                            resolution=1/LFP_sample_rate,
                            description="Probe A LFP data"))

nwbfile.add_acquisition(ElectricalSeries('Neuropix-PXI-100.ProbeB-AP',
                            data = np.zeros((num_samples,384)),
                            electrodes = probeB_AP_region,
                            timestamps=np.linspace(0,num_samples / AP_sample_rate, num_samples),
                            resolution=1/AP_sample_rate,
                            description="Probe A AP data"))

nwbfile.add_acquisition(ElectricalSeries('Neuropix-PXI-100.ProbeB-LFP',
                            data = np.zeros((num_samples // 12,384)),
                            electrodes = probeB_LFP_region,
                            timestamps=np.linspace(0,num_samples / LFP_sample_rate, num_samples),
                            resolution=1/LFP_sample_rate,
                            description="Probe B LFP data"))

In [4]:
from pynwb.ecephys import EventWaveform, SpikeEventSeries

event_waveform = EventWaveform(name='SpikeDetector-101.ProbeA-AP.spikes')

electrode_table_region = nwbfile.create_electrode_table_region([0], 'Single electrode')

num_spikes = 100

spike_data = np.zeros((num_spikes, 1, 60))
spike_timestamps = np.random.rand(num_spikes) * 1/30

spike_event_series = SpikeEventSeries(name='SingleElectrode1',
                            data = spike_data,
                            timestamps =spike_timestamps,
                            electrodes =electrode_table_region,
                            conversion=0.195,
                            description="Spike event series for a single electrode",
                            control=np.zeros((num_spikes,)),
                            control_description='unit_id')

event_waveform.add_spike_event_series(spike_event_series)

electrode_table_region = nwbfile.create_electrode_table_region([1], 'Single electrode')

spike_event_series = SpikeEventSeries(name='SingleElectrode2',
                            data = spike_data,
                            timestamps =spike_timestamps,
                            electrodes =electrode_table_region,
                            conversion=0.195,
                            description="Spike event series for a single electrode",
                            control=np.zeros((num_spikes,)),
                            control_description='unit_id')

event_waveform.add_spike_event_series(spike_event_series)

nwbfile.add_acquisition(event_waveform)

In [5]:
from pynwb.misc import IntervalSeries, AnnotationSeries

data = [1, -1, 1, -1]
timestamps = np.linspace(0,1/30,4)

interval_series = IntervalSeries(name='Neuropix-PXI-100.ProbeA-AP.TTL',
                            data = data,
                            timestamps =timestamps,
                            comments="This is data for a TTL channel",
                            description="TTL events for Probe A AP")

nwbfile.add_acquisition(interval_series)

interval_series = IntervalSeries(name='Neuropix-PXI-100.ProbeB-AP.TTL',
                            data = data,
                            timestamps =timestamps,
                            comments="This is data for a TTL channel",
                            description="TTL events for Probe B AP")

nwbfile.add_acquisition(interval_series)

annotation_series = AnnotationSeries(name='messages',
                            data = ['Example message', 'Another message', 'Yet another message'],
                            timestamps =[0.001, 0.002, 0.003],
                            description="Open Ephys GUI global messages")

nwbfile.add_acquisition(annotation_series)

In [6]:
from pynwb import NWBHDF5IO

io = NWBHDF5IO('oe_neuropixels_example.nwb', mode='w')
io.write(nwbfile)
io.close()

/Users/franman/miniconda3/envs/pynwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'SpikeEventSeries/control': Value with data type float64 is being converted to data type uint64 (min specification: uint8).
  warnings.warn(full_warning_msg, DtypeConversionWarning)


In [7]:
io = NWBHDF5IO('oe_neuropixels_example.nwb', 'r')
nwbfile_in = io.read()

In [8]:
test_timeseries_in = nwbfile_in.acquisition['Neuropix-PXI-100.ProbeA-LFP']
print(test_timeseries_in)

Neuropix-PXI-100.ProbeA-LFP pynwb.ecephys.ElectricalSeries at 0x4726449056
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (100, 384), type "<f8">
  description: Probe A LFP data
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  resolution: 0.0004
  timestamps: <HDF5 dataset "timestamps": shape (1200,), type "<f8">
  timestamps_unit: seconds
  unit: volts



In [9]:
print(test_timeseries_in.data[:])
io.close()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
import h5py as h5

In [11]:
f = h5.File('oe_neuropixels_example.nwb', 'r')

In [12]:
f.keys()

<KeysViewHDF5 ['acquisition', 'analysis', 'file_create_date', 'general', 'identifier', 'processing', 'session_description', 'session_start_time', 'specifications', 'stimulus', 'timestamps_reference_time']>

In [13]:
GROUP_TYPE = h5._hl.group.Group
DATASET_TYPE = h5._hl.dataset.Dataset

In [14]:
def printHDF5(obj, key, level):
    
    if type(obj[key]) == GROUP_TYPE:
        print(''.join(['  ']*level) + key)
        for k in obj[key].keys():
            printHDF5(obj[key], k, level+1)
    elif type(obj[key]) == DATASET_TYPE:
        print(''.join(['  ']*(level)) + key + ': ' + str(obj[key].shape))

In [15]:
for key in f.keys():
    printHDF5(f, key, 0)

acquisition
  Neuropix-PXI-100.ProbeA-AP
    data: (1200, 384)
    electrodes: (384,)
    timestamps: (1200,)
  Neuropix-PXI-100.ProbeA-AP.TTL
    data: (4,)
    timestamps: (4,)
  Neuropix-PXI-100.ProbeA-LFP
    data: (100, 384)
    electrodes: (384,)
    timestamps: (1200,)
  Neuropix-PXI-100.ProbeB-AP
    data: (1200, 384)
    electrodes: (384,)
    timestamps: (1200,)
  Neuropix-PXI-100.ProbeB-AP.TTL
    data: (4,)
    timestamps: (4,)
  Neuropix-PXI-100.ProbeB-LFP
    data: (100, 384)
    electrodes: (384,)
    timestamps: (1200,)
  SpikeDetector-101.ProbeA-AP.spikes
    SingleElectrode1
      control: (100,)
      control_description: (7,)
      data: (100, 1, 60)
      electrodes: (1,)
      timestamps: (100,)
    SingleElectrode2
      control: (100,)
      control_description: (7,)
      data: (100, 1, 60)
      electrodes: (1,)
      timestamps: (100,)
  messages
    data: (3,)
    timestamps: (3,)
analysis
file_create_date: (1,)
general
  devices
    ProbeA
    ProbeB
  expe

In [16]:
f['general']['extracellular_ephys']['electrodes']['id']

<HDF5 dataset "id": shape (1536,), type "<i8">

In [17]:
f.close()